> **⚠️ Important**: Run cells in order, as each cell depends on the previous state.


# Tutorial: End-to-End Research Workflow with Theta Sweep Pipeline> **Reading Time**: ~30-35 minutes> **Difficulty**: Beginner to Intermediate> **Prerequisites**: Basic understanding of navigation trajectoriesThis tutorial demonstrates how to use the ThetaSweepPipeline for complete end-to-end analysis of experimental trajectory data without needing detailed knowledge of CANN implementation.---

## Table of Contents1. [Introduction to Pipelines](#1-introduction-to-pipelines)2. [Quick Start: Basic Pipeline Usage](#2-quick-start-basic-pipeline-usage)3. [Loading External Trajectory Data](#3-loading-external-trajectory-data)4. [Advanced Customization](#4-advanced-customization)5. [Complete Research Example](#5-complete-research-example)6. [Next Steps](#6-next-steps)---

## 1. Introduction to Pipelines### 1.1 What is a Pipeline?A **pipeline** provides a high-level interface for complete analyses without requiring implementation details:

In [ ]:
# Traditional approach (manual)1. Load data2. Create networks3. Run simulation4. Compute analyses5. Generate visualizations6. Save results# Pipeline approach (automated)pipeline = ThetaSweepPipeline(trajectory_data, times)results = pipeline.run()  # Everything done!

### 1.2 Why Use Pipelines?**For experimental neuroscientists**:- No need to understand CANN mathematics- Focus on your data and questions- Reproducible, standardized analyses**For computational researchers**:- Rapid prototyping- Parameter sweeps and batch processing- Consistent output formats### 1.3 ThetaSweepPipeline OverviewThe `ThetaSweepPipeline` implements complete theta sweep analysis:**Inputs**:- Trajectory data (position over time)- Timing information**Automatic processing**:- Direction cell network simulation- Grid cell network simulation- Theta modulation computation- Population activity analysis**Outputs**:- Trajectory analysis plots- Theta sweep animations- Population activity visualizations- Raw simulation data for custom analysis---

## 2. Quick Start: Basic Pipeline Usage### 2.1 Minimal ExampleThe simplest possible usage - just trajectory and times:

In [ ]:
import numpy as npfrom canns.pipeline import ThetaSweepPipeline# Example: Load your experimental trajectory# positions: shape (n_steps, 2) - [x, y] coordinates# times: shape (n_steps,) - timestamps in secondspositions = np.load('my_trajectory.npy')  # Your datatimes = np.load('my_times.npy')# Run complete analysis (one line!)pipeline = ThetaSweepPipeline(    trajectory_data=positions,    times=times)results = pipeline.run(output_dir="results/")print(f"Animation saved to: {results['animation_path']}")print(f"Analysis plots in: results/")

That's it! The pipeline handles everything automatically.### 2.2 Understanding the OutputsAfter running, you'll find in the output directory:```results/├── trajectory_analysis.png       # Trajectory path and statistics├── population_activity_hd.png    # Direction cell activity├── population_activity_gc.png    # Grid cell activity├── theta_sweep_animation.gif     # Complete dynamics animation└── simulation_data.npz           # Raw data for custom analysis```### 2.3 Quick Data Format CheckYour trajectory data should be:

In [ ]:
# positions: (n_steps, 2) arrayprint(f"Position shape: {positions.shape}")  # Should be (N, 2)print(f"Position range X: [{positions[:,0].min()}, {positions[:,0].max()}]")print(f"Position range Y: [{positions[:,1].min()}, {positions[:,1].max()}]")# times: (n_steps,) arrayprint(f"Times shape: {times.shape}")  # Should be (N,)print(f"Duration: {times[-1] - times[0]:.2f}s")print(f"Mean dt: {np.mean(np.diff(times)):.4f}s")

**Common issues**:- Positions not in meters? Scale appropriately- Times not in seconds? Convert first- Non-uniform sampling? Pipeline handles automatically---

## 3. Loading External Trajectory Data### 3.1 From CSV Files

In [ ]:
import pandas as pd# Load from CSVdf = pd.read_csv('trajectory.csv')# Extract positions and timespositions = df[['x', 'y']].values  # (n_steps, 2)times = df['time'].values          # (n_steps,)# Run pipelinepipeline = ThetaSweepPipeline(positions, times)results = pipeline.run()

**CSV format example**:```csvtime,x,y0.000,0.5,0.50.001,0.501,0.5020.002,0.503,0.505...```### 3.2 From MATLAB Files

In [ ]:
from scipy.io import loadmat# Load MATLAB filedata = loadmat('trajectory.mat')positions = data['positions']  # Already (n_steps, 2)times = data['times'].flatten()  # Flatten if neededpipeline = ThetaSweepPipeline(positions, times)results = pipeline.run()

### 3.3 From Tracking SoftwareCommon formats from tracking systems:

In [ ]:
# DeepLabCut outputimport pandas as pddlc_data = pd.read_csv('tracking_output.csv', header=[0,1,2])x = dlc_data[('bodypart1', 'x')].valuesy = dlc_data[('bodypart1', 'y')].valuespositions = np.column_stack([x, y])# Bonsai outputbonsai_data = pd.read_csv('bonsai_tracking.csv')positions = bonsai_data[['X', 'Y']].values / 100  # Convert cm to m# Custom tracking# Always ensure: positions in meters, times in seconds

### 3.4 Synthetic Test DataFor testing or demonstration:

In [ ]:
def create_test_trajectory(n_steps=1000, dt=0.002):    """Create smooth test trajectory"""    times = np.linspace(0, (n_steps-1)*dt, n_steps)    # Circular trajectory with some noise    t_param = np.linspace(0, 4*np.pi, n_steps)    radius = 0.5    center = np.array([0.75, 0.75])    x = center[0] + radius * np.cos(t_param)    y = center[1] + radius * np.sin(t_param)    # Add small noise    noise = np.random.normal(0, 0.01, (n_steps, 2))    positions = np.column_stack([x, y]) + noise    return positions, times# Test the pipelinepositions, times = create_test_trajectory()pipeline = ThetaSweepPipeline(positions, times, env_size=1.5)results = pipeline.run(output_dir="test_results/")

---

## 4. Advanced Customization### 4.1 Environment ConfigurationAdjust environment parameters:

In [ ]:
pipeline = ThetaSweepPipeline(    trajectory_data=positions,    times=times,    env_size=2.0,    # Environment size (meters)    dt=0.001,        # Simulation time step (seconds))

**When to adjust**:- `env_size`: Match your experimental arena (1m, 2m, etc.)- `dt`: Finer time resolution for very fast movements### 4.2 Network ParametersCustomize direction and grid cell networks:

In [ ]:
pipeline = ThetaSweepPipeline(    trajectory_data=positions,    times=times,    # Direction cell network    direction_cell_params={        'num': 100,                  # Number of direction cells        'adaptation_strength': 15.0,  # SFA strength        'noise_strength': 0.0,       # Activity noise    },    # Grid cell network    grid_cell_params={        'num_gc_x': 100,            # Grid cells per dimension        'adaptation_strength': 8.0,  # SFA strength        'mapping_ratio': 5,          # Grid spacing control        'phase_offset': 1.0/20,      # Theta sweep magnitude    },)

**Parameter effects**:- Higher `adaptation_strength`: Stronger theta oscillations- Larger `mapping_ratio`: Smaller grid spacing- Larger `phase_offset`: Bigger theta sweeps### 4.3 Theta Modulation SettingsControl theta rhythm parameters:

In [ ]:
pipeline = ThetaSweepPipeline(    trajectory_data=positions,    times=times,    theta_params={        'theta_strength_hd': 1.0,    # HD cell modulation strength        'theta_strength_gc': 0.5,    # Grid cell modulation strength        'theta_cycle_len': 100.0,    # Cycle length (ms)    },)

**Biological correspondence**:- `theta_cycle_len=100ms` → 10 Hz theta frequency- `theta_cycle_len=125ms` → 8 Hz theta frequency### 4.4 Animation and VisualizationCustomize output visualizations:

In [ ]:
results = pipeline.run(    output_dir="custom_results/",    save_animation=True,           # Generate animation    save_plots=True,               # Save analysis plots    animation_fps=10,              # Animation frame rate    animation_n_step=20,           # Sample every N frames    verbose=True,                  # Print progress)

**Performance tips**:- Lower `animation_fps` for smaller files- Higher `animation_n_step` for faster generation- Set `save_animation=False` to skip animation (faster)---

## 5. Complete Research Example### 5.1 Realistic ScenarioAnalyzing a recorded navigation session:

In [ ]:
import numpy as npimport pandas as pdfrom canns.pipeline import ThetaSweepPipeline# Load experimental dataprint("Loading experimental trajectory...")df = pd.read_csv('experiment_2024_session_3.csv')# Extract and preprocesspositions = df[['x_cm', 'y_cm']].values / 100  # Convert cm to meterstimes = df['timestamp_ms'].values / 1000       # Convert ms to seconds# Data quality checkprint(f"Trajectory: {len(positions)} samples")print(f"Duration: {times[-1] - times[0]:.2f}s")print(f"Position range: X[{positions[:,0].min():.2f}, {positions[:,0].max():.2f}]m, "      f"Y[{positions[:,1].min():.2f}, {positions[:,1].max():.2f}]m")# Remove any NaN values (tracking failures)valid = ~np.isnan(positions).any(axis=1)positions = positions[valid]times = times[valid]print(f"Valid samples: {len(positions)}")

### 5.2 Run Comprehensive Analysis

In [ ]:
# Configure pipeline with experimental parameterspipeline = ThetaSweepPipeline(    trajectory_data=positions,    times=times,    env_size=1.5,  # 1.5m x 1.5m arena    direction_cell_params={        'num': 100,        'adaptation_strength': 15.0,        'noise_strength': 0.05,  # Small noise for realism    },    grid_cell_params={        'num_gc_x': 100,        'adaptation_strength': 8.0,        'mapping_ratio': 5,    },    theta_params={        'theta_strength_hd': 1.0,        'theta_strength_gc': 0.5,        'theta_cycle_len': 100.0,  # 10 Hz    },)# Run analysisprint("\nRunning theta sweep analysis...")results = pipeline.run(    output_dir="experiment_analysis/",    save_animation=True,    save_plots=True,    animation_fps=15,    verbose=True,)print("\n✅ Analysis complete!")print(f"📊 Results saved to: experiment_analysis/")print(f"🎬 Animation: {results['animation_path']}")

### 5.3 Custom Post-ProcessingAccess raw simulation data for custom analyses:

In [ ]:
# Load simulation resultssim_data = results['simulation_data']# Available datadc_activity = sim_data['dc_activity']    # Direction cell firinggc_activity = sim_data['gc_activity']    # Grid cell firingtheta_phase = sim_data['theta_phase']    # Theta phase over timeinternal_pos = sim_data['internal_position']  # Decoded position# Example analysis: Phase precessionimport matplotlib.pyplot as plt# Find high activity periodsactivity_threshold = gc_activity.mean() + gc_activity.std()high_activity = gc_activity.max(axis=1) > activity_threshold# Plot phase vs time for high activityplt.figure(figsize=(12, 4))plt.scatter(times[high_activity], theta_phase[high_activity],            c=gc_activity[high_activity].max(axis=1),            cmap='viridis', s=10, alpha=0.6)plt.xlabel('Time (s)')plt.ylabel('Theta Phase (rad)')plt.title('Phase Precession During High Grid Cell Activity')plt.colorbar(label='Max Activity')plt.savefig('experiment_analysis/phase_precession.png', dpi=150)plt.show()# Compare decoded vs actual positionplt.figure(figsize=(10, 5))plt.subplot(1, 2, 1)plt.plot(positions[:,0], positions[:,1], 'k-', alpha=0.5, label='Actual')plt.plot(internal_pos[:,0], internal_pos[:,1], 'r-', alpha=0.7, label='Decoded')plt.xlabel('X Position (m)')plt.ylabel('Y Position (m)')plt.title('Position Tracking Accuracy')plt.legend()plt.axis('equal')plt.subplot(1, 2, 2)error = np.linalg.norm(positions - internal_pos, axis=1)plt.plot(times, error, 'b-', alpha=0.7)plt.xlabel('Time (s)')plt.ylabel('Position Error (m)')plt.title('Decoding Error Over Time')plt.tight_layout()plt.savefig('experiment_analysis/position_accuracy.png', dpi=150)plt.show()print(f"\n📈 Mean position error: {error.mean():.3f}m")print(f"📈 Max position error: {error.max():.3f}m")

### 5.4 Batch Processing Multiple SessionsProcess multiple experimental sessions:

In [ ]:
import globfrom pathlib import Path# Find all session filessession_files = glob.glob('experiments/session_*.csv')print(f"Found {len(session_files)} sessions to process")# Process each sessionfor session_file in session_files:    session_name = Path(session_file).stem    print(f"\nProcessing {session_name}...")    # Load data    df = pd.read_csv(session_file)    positions = df[['x_cm', 'y_cm']].values / 100    times = df['timestamp_ms'].values / 1000    # Run pipeline    pipeline = ThetaSweepPipeline(positions, times, env_size=1.5)    output_dir = f"batch_results/{session_name}/"    results = pipeline.run(        output_dir=output_dir,        save_animation=False,  # Skip animation for speed        save_plots=True,        verbose=False,    )    print(f"  ✓ Complete: {output_dir}")print("\n🎉 Batch processing complete!")

---

## 6. Next StepsCongratulations! You've learned how to use the ThetaSweepPipeline for end-to-end theta sweep analysis.### Key Takeaways1. **Pipelines simplify workflows** - One-line analysis of complex models2. **Flexible data loading** - CSV, MATLAB, tracking software3. **Automatic outputs** - Plots, animations, and raw data4. **Customizable parameters** - Full control when needed5. **Batch processing** - Analyze multiple sessions efficiently### When to Use Pipelines**Perfect for**:- Experimental neuroscientists without coding expertise- Rapid prototyping and exploratory analysis- Standardized processing of multiple datasets- Publication-quality figure generation- Teaching and demonstrations**Consider manual approach when**:- Need non-standard model architectures- Implementing new analysis methods- Fine-grained control over every step- Extending the pipeline functionality### Pipeline Features Summary| Feature | Basic Usage | Advanced Usage ||---------|-------------|----------------|| Data input | `trajectory_data, times` | Multiple format loaders || Environment | Auto-detected | Customizable `env_size`, `dt` || Networks | Default parameters | Full parameter dictionaries || Theta | Default 10 Hz | Custom frequency, strength || Outputs | Standard plots | Raw data + custom analysis || Batch | Single session | Multi-session processing |### Continue Learning- **Related**: [Tutorials 1-7](../scenario_1_cann_modeling/01_build_cann_model.ipynb) - Understanding models in detail- **Related**: [Scenario 3](../scenario_3_brain_inspired_learning/01_pattern_storage_recall.ipynb) - Brain-inspired learning### Extending the PipelineWant to modify or extend the pipeline?1. **Check source code**: `canns.pipeline.theta_sweep.ThetaSweepPipeline`2. **Inherit and customize**:   ```python   from canns.pipeline import ThetaSweepPipeline   class MyCustomPipeline(ThetaSweepPipeline):       def custom_analysis(self):           # Add your analysis here           pass   ```3. **Contribute**: Submit enhancements via GitHub pull requests### Getting Help- **Documentation**: [https://canns.readthedocs.io](https://canns.readthedocs.io)- **Examples**: See `examples/pipeline/` directory- **Issues**: [GitHub Issues](https://github.com/PKU-NIP-Lab/CANNs/issues)- **Community**: Discussions and Q&A on GitHub### Best Practices1. **Data quality first**: Clean tracking data before pipeline2. **Start simple**: Use default parameters initially3. **Validate outputs**: Check trajectory plots for sanity4. **Document parameters**: Save configuration for reproducibility5. **Version control**: Track pipeline version used for each analysisThank you for completing the CANN tutorial series! You now have comprehensive knowledge of CANN modeling, brain-inspired learning, and practical research workflows.